<a href="https://colab.research.google.com/github/vijaygwu/SEAS8210/blob/main/Embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IDs → embeddings lookup, cosine similarity, and sinusoidal positions.
# No external packages required besides numpy.

import re
import math
from collections import Counter, defaultdict
import numpy as np

np.random.seed(13)  # deterministic demo

def normalize(text):
    # Minimal normalization: collapse whitespace and ensure basic ASCII apostrophe normalization
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')
    text = re.sub(r"\s+", " ", text.strip())
    return text

def whitespace_tokenize(text):
    # Simple heuristic tokenizer splitting punctuation
    text = normalize(text)
    # separate punctuation
    text = re.sub(r"([.,!?;:()\[\]{}])", r" \1 ", text)
    # split contractions: can't -> can n't
    text = re.sub(r"(\w)('t|'ll|'re|'ve|'s|'d|'m)\b", r"\1 \2", text)
    toks = [t for t in text.split(" ") if t]
    return toks

def char_tokenize(text):
    text = normalize(text)
    # Prefix each word with a "▁" marker like SentencePiece
    words = text.split(" ")
    chars = []
    for w in words:
        if not w: continue
        chars.append("▁")
        chars.extend(list(w))
    return chars

# ---- Tiny BPE trainer/encoder (character level → subword merges) ----

def get_vocab_from_corpus(texts):
    """Return list of words split as character sequences with a '▁' prefix per word."""
    vocab = []
    for t in texts:
        t = normalize(t)
        for w in t.split(" "):
            if not w:
                continue
            vocab.append(["▁"] + list(w))
    return vocab

def count_pair_stats(vocab):
    """Count adjacent pair frequencies across all tokens in vocab."""
    pairs = Counter()
    for token_list in vocab:
        for i in range(len(token_list)-1):
            pairs[(token_list[i], token_list[i+1])] += 1
    return pairs

def merge_vocab(vocab, pair_to_merge):
    """Merge chosen pair in all token lists."""
    a, b = pair_to_merge
    new_vocab = []
    for token_list in vocab:
        i = 0
        merged = []
        while i < len(token_list):
            if i < len(token_list)-1 and token_list[i] == a and token_list[i+1] == b:
                merged.append(a+b)
                i += 2
            else:
                merged.append(token_list[i])
                i += 1
        new_vocab.append(merged)
    return new_vocab

def train_bpe(texts, num_merges=50):
    vocab = get_vocab_from_corpus(texts)
    merges = []
    for _ in range(num_merges):
        stats = count_pair_stats(vocab)
        if not stats:
            break
        pair, freq = stats.most_common(1)[0]
        # Stop if merging doesn't change anything meaningful
        if freq < 2:
            break
        merges.append(pair)
        vocab = merge_vocab(vocab, pair)
    # Build a ranking for merges (earlier is higher priority)
    merge_ranks = {pair: i for i, pair in enumerate(merges)}
    return merges, merge_ranks

def bpe_encode_word(word, merge_ranks):
    """Encode a single word by greedily applying highest-priority merges present."""
    # Start from char-level with a prefixed whitespace marker
    tokens = ["▁"] + list(word)
    # Keep merging while any adjacent pair is in merge_ranks
    while True:
        candidates = []
        for i in range(len(tokens)-1):
            pair = (tokens[i], tokens[i+1])
            if pair in merge_ranks:
                candidates.append((merge_ranks[pair], i, pair))
        if not candidates:
            break
        # Highest priority = smallest rank
        _, i, pair = min(candidates, key=lambda x: x[0])
        a, b = pair
        tokens = tokens[:i] + [a+b] + tokens[i+2:]
    return tokens

def bpe_tokenize(text, merge_ranks):
    text = normalize(text)
    out = []
    for w in text.split(" "):
        if not w: continue
        out.extend(bpe_encode_word(w, merge_ranks))
    return out

# ---- Demo content ----

train_texts = [
    "low lower lowest",
    "new newer newest",
    "I can't believe it's not butter",
    "unbelievable unbeliever believable believing",
    "Tokenizer tokenization tokens tokenized tokenizing",
    "cats catlike dogs doglike running runner runs"
]

print("=== Training a tiny BPE on a small classroom corpus ===")
merges, merge_ranks = train_bpe(train_texts, num_merges=80)
print(f"Learned {len(merges)} merges. First 15 merges:")
for i, p in enumerate(merges[:15], start=1):
    print(f"{i:>2}: {p[0]} + {p[1]} → {p[0]+p[1]}")

sample = "I can't believe it's not butter and unbelievable newness"
print("\n=== Sample sentence ===")
print(sample)

print("\nWhitespace tokens:")
print(whitespace_tokenize(sample))

print("\nCharacter tokens with '▁' markers:")
print(char_tokenize(sample))

print("\nTiny BPE tokens (learned from corpus):")
bpe_toks = bpe_tokenize(sample, merge_ranks)
print(bpe_toks)

# ---- Map tokens to IDs (build a toy vocab from BPE on training texts) ----
print("\n=== Building a toy vocabulary from BPE over training texts ===")

vocab_set = set()
for t in train_texts:
    for tok in bpe_tokenize(t, merge_ranks):
        vocab_set.add(tok)

# Add specials
specials = ["<PAD>", "<BOS>", "<EOS>", "<UNK>"]
vocab = specials + sorted(vocab_set)
token_to_id = {tok: i for i, tok in enumerate(vocab)}
id_to_token = {i: tok for tok, i in token_to_id.items()}
print(f"Vocab size: {len(vocab)}")

# Encode the sample to IDs
def to_ids(tokens):
    return [token_to_id.get(t, token_to_id["<UNK>"]) for t in tokens]

ids = to_ids(bpe_toks)
print("IDs:", ids)

# ---- Embeddings: lookup & cosine similarity ----
d_model = 16
E = (np.random.randn(len(vocab), d_model) / np.sqrt(d_model)).astype(np.float32)

def embed(ids):
    return E[ids]  # [seq_len, d_model]

def cosine(a, b):
    a = a / (np.linalg.norm(a) + 1e-9)
    b = b / (np.linalg.norm(b) + 1e-9)
    return float(np.dot(a, b))

print("\n=== Embedding lookup ===")
print(f"E.shape = {E.shape}  # [vocab_size, d_model]")

# Pick a few tokens to compare
probe_tokens = ["▁new", "▁low", "er", "est", "un", "able"]
probe_ids = [token_to_id.get(t, token_to_id["<UNK>"]) for t in probe_tokens]
for t, i in zip(probe_tokens, probe_ids):
    vec = E[i]
    print(f"Token {t!r} → ID {i} → first 4 dims {vec[:4].round(3)}")

print("\nCosine similarities among probes (random init; for illustration):")
for i, ti in enumerate(probe_tokens):
    for j in range(i+1, len(probe_tokens)):
        tj = probe_tokens[j]
        sim = cosine(E[token_to_id.get(ti, token_to_id['<UNK>'])],
                     E[token_to_id.get(tj, token_to_id['<UNK>'])])
        print(f"cos({ti:>5}, {tj:>5}) = {sim:+.3f}")

# ---- Positional encodings (sin/cos) ----
def sinusoidal_positions(max_len, d_model):
    pe = np.zeros((max_len, d_model), dtype=np.float32)
    position = np.arange(0, max_len, dtype=np.float32)[:, None]
    div_term = np.exp(np.arange(0, d_model, 2, dtype=np.float32) * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    return pe

print("\n=== Sinusoidal positional encodings (first 3 positions, 8 dims) ===")
pe = sinusoidal_positions(3, 8)
np.set_printoptions(precision=3, suppress=True)
print(pe)

print("\n=== Putting it together (toy forward input) ===")
seq_emb = embed(ids[:10])
pe10 = sinusoidal_positions(len(seq_emb), d_model)
x0 = seq_emb + pe10  # what goes into the first transformer block (toy illustration)
print("x0 shape:", x0.shape)
print("First token embedding+pos (first 4 dims):", x0[0][:4].round(3))


=== Training a tiny BPE on a small classroom corpus ===
Learned 40 merges. First 15 merges:
 1: l + i → li
 2: k + e → ke
 3: e + r → er
 4: b + e → be
 5: be + li → beli
 6: beli + e → belie
 7: belie + v → believ
 8: u + n → un
 9: o + ke → oke
10: oke + n → oken
11: ▁ + n → ▁n
12: oken + i → okeni
13: okeni + z → okeniz
14: ▁ + t → ▁t
15: ▁ + l → ▁l

=== Sample sentence ===
I can't believe it's not butter and unbelievable newness

Whitespace tokens:
['I', 'can', "'t", 'believe', 'it', "'s", 'not', 'butter', 'and', 'unbelievable', 'newness']

Character tokens with '▁' markers:
['▁', 'I', '▁', 'c', 'a', 'n', "'", 't', '▁', 'b', 'e', 'l', 'i', 'e', 'v', 'e', '▁', 'i', 't', "'", 's', '▁', 'n', 'o', 't', '▁', 'b', 'u', 't', 't', 'e', 'r', '▁', 'a', 'n', 'd', '▁', 'u', 'n', 'b', 'e', 'l', 'i', 'e', 'v', 'a', 'b', 'l', 'e', '▁', 'n', 'e', 'w', 'n', 'e', 's', 's']

Tiny BPE tokens (learned from corpus):
['▁', 'I', '▁ca', 'n', "'", 't', '▁believ', 'e', '▁', 'i', 't', "'", 's', '▁n', 'o', 't'